In [1]:
import importlib
import pandas as pd
import numpy as np
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
""" 
Для формування списку ЄДРПОУ навчальних закладів, які потрібно наносити на мапу, використані бази даних МОН, опубліковані у листопаді 2019 року на Порталі відкритих даних.

### Перелік закладів загальної середньої та дошкільної освіти (ЗЗСДО)
    https://data.gov.ua/dataset/bd70757a-ab49-46be-a650-608a9b3bd7b1
    
### Перелік закладів позашкільної освіти (державна форма власності)
    https://data.gov.ua/dataset/9a88fb96-a043-4769-acf1-1c4a948ec285
    Перелік закладів позашкільної освіти системи Міністерства освіти і науки України
    https://data.gov.ua/dataset/18d7fb70-3781-4017-88d2-651e14d14211
    Заклади професійної (професійно-технічної) освіти
    https://data.gov.ua/dataset/824513a9-aa5e-45c8-b45e-dced6909761a
"""

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [3]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns.tolist()
print("Дані з файлу викачки BIPro Prozorro: " + "df_raw" + str(df_raw.shape))

Дані з файлу викачки BIPro Prozorro: df_raw(537893, 11)


##### Step 0: Підготовка бази МОН для використання у фільтруванні закупівель

In [4]:
expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';', low_memory = False)
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';', low_memory = False)

In [5]:
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])
print("expdata_sc", expdata_sc.shape)

expdata_sc (15206, 21)


In [6]:
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])
print("expdata_dnz", expdata_dnz.shape)

expdata_dnz (15745, 21)


In [7]:
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)
print("База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data", df_mon_data.shape)

База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data (30951, 22)


In [8]:
filter_keywords_exclude = ["департамент", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_raw[df_raw['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_exclude)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()
print("edrpou_upravlin_osvity_prozorro", len(edrpou_upravlin_osvity_prozorro))

mask = (df_mon_data['edrpou'].isin(edrpou_upravlin_osvity_prozorro))
df_mon_data_bez_upravlin_osvity = df_mon_data.loc[~mask]
print("df_mon_data_bez_upravlin_osvity", df_mon_data_bez_upravlin_osvity.shape)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


edrpou_upravlin_osvity_prozorro 234
df_mon_data_bez_upravlin_osvity (30881, 22)


In [ ]:
"""
TODO:
    Артем: 
        створити базу управлінь освіти для формування переліку закупівель управлінь освіти 
        (закупівлі можуть дублюватись із закупівлями шкіл і садків)
        ЄДРПОУ - Місто (для Києва - по районам, якщо можливо)

"""

##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro. Виділення закупівель шкіл-замовників , згідно з базою МОН (пошук по колонці "IDOrganizator"

In [9]:
mon_data = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()
# mon_data = get_mon_data()

df_tenders_by_schools_by_edrpou = df_raw[df_raw['IDOrganizator'].isin(mon_data)]
df_undefined_1 = df_raw[~df_raw['IDOrganizator'].isin(mon_data)]

print("Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou " + str(df_tenders_by_schools_by_edrpou.shape))
print("Невизначені записи: df_undefined_1 " + str(df_undefined_1.shape))

Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou (32188, 11)
Невизначені записи: df_undefined_1 (505705, 11)


In [ ]:
# evaluation of results of Step 1

# df_tenders_by_schools_by_edrpou['Организатор'].value_counts()

In [11]:
# # Experiments Step 1

# school_names = df_mon_data_bez_upravlin_osvity.university_name.tolist()
# df_tenders_by_schools_by_name = df_raw[df_raw['Организатор'].isin(school_names)]
# df_tenders_by_schools_by_name.shape

# print(df_mon_data_bez_upravlin_osvity.columns)

# cols = ['edrpou', 'university_name', 'university_type_name', 'education_type_name']
# df_mon_data_bez_upravlin_osvity_for_merge = df_mon_data_bez_upravlin_osvity[cols]

# df_tenders_by_schools_by_edrpou_extended = pd.merge(df_tenders_by_schools_by_edrpou, df_mon_data_bez_upravlin_osvity_merge, 
#                                                                                  left_on='IDOrganizator',
#                                                                                  right_on='edrpou',
#                                                                                  how='left') 
# df_tenders_by_schools_by_edrpou_extended.sample(3)

##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [12]:
df_tenders_by_schools_by_keywords = df_undefined_1[df_undefined_1['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), case = False, na = False, regex = True) ]
print(f"Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords {df_tenders_by_schools_by_keywords.shape}")

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords (9351, 11)


In [ ]:
# evalutaion #1 of results of Step 2
# df_tenders_by_schools_by_keywords['Организатор'].value_counts()

In [13]:
# solution for Step 2

filter_keywords_step_2_exclude = ['професійн', 'міжшкільн']
df_tenders_by_schools_by_keywords_2_temp = df_tenders_by_schools_by_keywords[~df_tenders_by_schools_by_keywords['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_exclude)), case = False, na = False, regex = True)]
filter_keywords_step_2_include = ["школа І-І.*ступенів", "навчальн.*виховн.*комплекс", "гімназ", "ліце", "дошкільний"]
df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords_2_temp[df_tenders_by_schools_by_keywords_2_temp['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_include)), case = False, na = False, regex = True)]
print(f"Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів, {df_tenders_by_schools_by_keywords_2_temp.shape} \nДодаткове фільтрування датафрейму: {filter_keywords_step_2_include}. Залишилось:, {df_tenders_by_schools_by_keywords_2.shape}")

Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів, (6890, 11) 
Додаткове фільтрування датафрейму: ['школа І-І.*ступенів', 'навчальн.*виховн.*комплекс', 'гімназ', 'ліце', 'дошкільний']. Залишилось:, (329, 11)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [ ]:
"""
TODO:
    Артем: 
        Перевірити детально, чи не видаляє filter_keywords_step_2_exclude потрібні закупівлі

"""

In [ ]:
# evalutaion #2 of results of Step 2

# df_tenders_by_schools_by_keywords_2['Организатор'].value_counts()

##### Step 2.5: Об'єднання двох датафреймів шкіл-замовників в один

In [14]:
df_tenders_by_schools = pandas.concat([df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_2])

print(f"Закупівлі шкіл-замовників: df_tenders_by_schools {df_tenders_by_schools.shape} \nНевизначені записи: df_undefined_1:{df_undefined_1.shape}")

Закупівлі шкіл-замовників: df_tenders_by_schools (32517, 11) 
Невизначені записи: df_undefined_1:(505705, 11)


##### Step 3: Класифікація шкіл-замовників по типах

In [15]:
columns_to_classify = ['Организатор']
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, columns_to_classify, filter_priority, filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:60: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

In [ ]:
# evaluation of Step 3

# pd.set_option("display.max_colwidth", 200)
# print("Заклади, де не визначений тип:")
# df_tenders_by_schools_classified[df_tenders_by_schools_classified['Тип закладу'] == 'Невідомо']['Организатор'].unique().tolist()

##### Step 4: Нумерація шкіл-замовників

In [16]:
columns_to_numerate = ['Организатор']
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, columns_to_numerate, reg_number)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Номер'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).apply(''.join)


##### Step 4.5: Найменування шкіл-замовників

In [17]:
columns_to_name = ['Организатор']
df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, columns_to_name, reg_quotes)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Назва'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).str[0].str[-1]


In [18]:
print('filter_poza', )
print('Було закупівель', df_tenders_by_schools_named.shape)
df_tenders_by_schools_named = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)),
                case = False, na = False, regex = True)]
df_tenders_by_schools_named['Номер'] = df_tenders_by_schools_named['Номер'].replace(r'^\s*$', np.nan, regex=True) # перетворити пусті строки в NaN
print(f'Залишилось закупівель після видалення закупівель закладів ключовими словами {filter_poza}, \ndf_tenders_by_schools_named', df_tenders_by_schools_named.shape)

filter_poza
Було закупівель (32517, 15)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


Залишилось закупівель після видалення закупівель закладів ключовими словами ['художн', 'мистецтв', 'дхш', 'дмш', 'музич', 'юнацьк.*спортивн', 'дюсш', 'дитяч.*спортивн'], 
df_tenders_by_schools_named (31859, 15)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
# # evaluation of Step 4

# for_analysis_step_4 = df_tenders_by_schools_named[df_tenders_by_schools_named['Назва'].isnull() & 
#                                   df_tenders_by_schools_named['Номер'].isnull()]
# print('Навчальні заклади, де не визначились номер чи назва:')
# for_analysis_step_4['Организатор'].unique().tolist()

##### Step 4.9: Форматування бази шкіл-замовників

In [19]:
df_tenders_by_schools = df_tenders_by_schools_named.copy()
df_tenders_by_schools['Заклад-замовник'] = True

col_names = training_data_columns.copy() 
col_names += ['Номер', 'Назва', 'Тип закладу', 'Заклад-замовник']

df_tenders_by_schools = df_tenders_by_schools[col_names]

##### Step 5: Виділення закупівель шкіл-не-замовників за допомогою пошуку ключових слів в колонках Тендер, ОписаниеТендера, Лот

In [20]:
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True)]

print("Закупівлі шкіл-не-замовників, визначених через Тендер, Лот, ОписаниеТендера: df_tenders_by_schools_by_keywords " + str(df_tenders_not_by_schools_by_keywords.shape))

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


Закупівлі шкіл-не-замовників, визначених через Тендер, Лот, ОписаниеТендера: df_tenders_by_schools_by_keywords (28708, 11)


In [ ]:
# evaluation of Step 5

# df_tenders_not_by_schools_by_keywords

##### Step 6: Класифікація шкіл-не-замовників по типах

In [21]:
# тимчасово винсені з filters_lists фільтри сюди для тестування порядковості

filter_priority = [
        filter_nvk,
        filter_gimnasium,
        filter_liceum,
        filter_specializovana,
        filter_internat,
        filter_kombinat,
        filter_sanatorna,
        filter_specializovana,
        filter_specialna,
        filter_sadik,
        filter_shkola 
    ]

filter_name_priority = [
        filter_nvk_name,
        filter_gimnasium_name,
        filter_liceum_name,
        filter_specializovana_name,
        filter_internat_name,
        filter_kombinat_name,
        filter_sanatorna_name,
        filter_specializovana_name,
        filter_specialna_name,
        filter_sadik_name,
        filter_shkola_name
    ]

In [22]:
columns_to_classify = ['Тендер', 'ОписаниеТендера', 'Лот']
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords, columns_to_classify, filter_priority, filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:40: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

In [ ]:
# # evaluation of Step 6

# pd.set_option("display.max_colwidth", 200)

# type_sample = 'Інтернат'
# df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] == type_sample][['Тендер', 'ОписаниеТендера', 'Лот', 'Тип закладу']].sort_values(by=['Тип закладу'])
# df_tenders_not_by_schools_classified['Тип закладу'].value_counts()

##### Step 7: Нумерація шкіл-не-замовників

In [23]:
"""
Принцип фільтрування в функції numeration():

    Визначається наявністья символу № в кожній колонці columns_to_numerate_step7
    
    Регулярний вираз знаходить всі паттерни і джойнить через кому
    
    Паралельно створюється колонка, де рахується кількість знайдених паттернів
    
    Результат фільтрування numeration() - потрапляє у датафрейм df_step7_numerated
"""
columns_to_numerate_step7 = ['Тендер', 'Лот']
action_mark = 'numeration'
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

df_step7_numerated = numeration(df_step7, columns_to_numerate_step7, reg_number, action_mark)
print("df_step7_numerated", df_step7_numerated.shape)

df_step7_numerated (23695, 19)


In [24]:
"""
    Якщо в колонках Лот і Тендер знайшло по одному паттерну і вони однакові - 
    закупівля вважається такою, де коректно визначений номер

    Ці закупівлі потрапляють до датафрейму df_step7_numerated_lot_tender

    Колонка "Номер" отримує знайдений номер "Лот_numeration"

    Решта закупівель потрапляють до датафрейму df_step7_numerated_undefined.
"""

df_step7_numerated['Номер'] = np.where(((df_step7_numerated['Тендер_numeration_count'] == 1) | 
                                        (df_step7_numerated['Лот_numeration_count'] == 1)) & 
                                        (df_step7_numerated['Лот_numeration_count'] == df_step7_numerated['Тендер_numeration_count']), 
                                        df_step7_numerated['Лот_numeration'], 
                                        'Невідомо')

df_step7_numerated_lot_tender = df_step7_numerated[df_step7_numerated['Номер'] != 'Невідомо']

df_step7_numerated_undefined = df_step7_numerated[df_step7_numerated['Номер'] == 'Невідомо']

print("df_step7_numerated_lot_tender", df_step7_numerated_lot_tender.shape, "\ndf_step7_numerated_undefined", df_step7_numerated_undefined.shape)

df_step7_numerated_lot_tender (17104, 20) 
df_step7_numerated_undefined (6591, 20)


In [25]:
"""
    Якщо в колонці Лот знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_lot
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_2
"""

df_step7_numerated_lot = df_step7_numerated_undefined.copy()

df_step7_numerated_lot['Номер'] = np.where(df_step7_numerated_lot['Лот_numeration_count'] == 1, 
                                                 df_step7_numerated_lot['Лот_numeration'], 
                                                 'Невідомо')

df_step7_numerated_undefined_2 = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] == 'Невідомо']
df_step7_numerated_lot = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] != 'Невідомо']

print("df_step7_numerated_lot", df_step7_numerated_lot.shape, "\ndf_step7_numerated_undefined_2", df_step7_numerated_undefined_2.shape)

df_step7_numerated_lot (527, 20) 
df_step7_numerated_undefined_2 (6064, 20)


In [26]:
"""
    Якщо в колонці Тендер знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_tender
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_3
"""
df_step7_numerated_tender = df_step7_numerated_undefined_2.copy()

df_step7_numerated_tender['Номер'] = np.where(df_step7_numerated_tender['Тендер_numeration_count'] == 1, 
                                                 df_step7_numerated_tender['Тендер_numeration'], 
                                                 'Невідомо')
df_step7_numerated_undefined_3 = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] == 'Невідомо']
df_step7_numerated_tender = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] != 'Невідомо']

print("df_step7_numerated_tender", df_step7_numerated_tender.shape, "\ndf_step7_numerated_undefined_3", df_step7_numerated_undefined_3.shape)

df_step7_numerated_tender (25, 20) 
df_step7_numerated_undefined_3 (6039, 20)


##### Step 7.5: Об'єднання датафреймів шкіл-не-замовників з номерами

In [27]:
frames = [df_step7_numerated_lot_tender, df_step7_numerated_lot, df_step7_numerated_tender]
df_tenders_not_by_schools_numerated = pd.concat(frames)
df_tenders_not_by_schools_numerated['Назва'] = np.nan # Створення пустої колонки "Назва" для подальшого обєднання з датафреймом з номерами

##### Step 8: Найменування шкіл-не-замовників

In [28]:
columns_to_classify_step8 = ['Тендер', 'Лот']
action_mark = 'naming'
df_step8 = df_step7_numerated_undefined_3.copy()

df_tenders_not_by_schools_named = naming(df_step8, columns_to_classify_step8, reg_quotes, action_mark)
print("df_tenders_not_by_schools_named", df_tenders_not_by_schools_named.shape)

df_tenders_not_by_schools_named (6039, 22)


In [29]:
df_tenders_not_by_schools_named['Назва'] = np.where((df_tenders_not_by_schools_named['Тендер_naming'].notnull()) & 
                             (df_tenders_not_by_schools_named['Лот_naming'].notnull()) & 
                             (df_tenders_not_by_schools_named['Тендер_naming'] == df_tenders_not_by_schools_named['Лот_naming']),
                             df_tenders_not_by_schools_named['Тендер_naming'], 
                             'Невідомо')

df_step8_undefined = df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Назва'] == 'Невідомо']
df_tenders_not_by_schools_named_lot_tender = df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Назва'] != 'Невідомо']

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [30]:
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)

In [31]:
df_tenders_not_by_schools['Заклад-замовник'] = False

In [32]:
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]
print("df_tenders_not_by_schools_numerated", df_tenders_not_by_schools_numerated.shape)

df_tenders_not_by_schools_numerated (17656, 21)


##### Step 9.5: Мердження Номеру і Назви у датафреймах у одну колонку Номер/Назва 
##### DO NOT RUN

In [52]:
# """
# У датафреймі Шкіл-замовників df_tenders_by_schools_temp в рядках, 
#     - де Номер не визначився, присвоюється Назва
#     - де Назва не визначилась, присвоюється поле Організатор
# """

# df_tenders_by_schools_temp = df_tenders_by_schools.copy()
# df_tenders_by_schools_temp['Номер/Назва'] = np.where(df_tenders_by_schools_temp['Номер'].notnull(), df_tenders_by_schools_temp['Номер'], df_tenders_by_schools_temp['Назва'])
# df_tenders_by_schools_temp['Номер/Назва'] = np.where(df_tenders_by_schools_temp['Номер/Назва'].isnull(), df_tenders_by_schools_temp['Организатор'], df_tenders_by_schools_temp['Номер/Назва'])

In [53]:
# """
# У датафреймі Шкіл-не-замовників df_tenders_not_by_schools в рядках, 
#     - де Номер не визначився, присвоюється Назва
#     - де Назва не визначилась, присвоюється поле Організатор

# В колонці 'Номер/Назва' значення "Невідомо" фактично означає закупівлю для кількох закладів освіти

# """

# df_tenders_not_by_schools_temp = df_tenders_not_by_schools.copy()
# df_tenders_not_by_schools_temp['Номер/Назва'] = np.where(df_tenders_not_by_schools_temp['Номер'].notnull(), df_tenders_not_by_schools_temp['Номер'], df_tenders_not_by_schools_temp['Назва'])

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [54]:
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)
print("df_tenders", df_tenders.shape)

df_tenders (55554, 15)


In [55]:
df_tenders_filtered_1 = df_tenders.sample(1000)
df_tenders_filtered_1['Correct'] = 'undefined'
print("df_tenders_filtered_1:", df_tenders_filtered_1.shape, "\nCols:", df_tenders_filtered_1.columns)
# df_tenders_filtered_1.to_csv('results/school-procurements-1000-random.csv', sep=';')

df_tenders_filtered_1: (1000, 16) 
Cols: Index(['ИдентификаторЛота', 'Идентификатор', 'Организатор', 'Тендер',
       'СуммаЛота', 'ОписаниеТендера', 'Лот', 'Адрес поставки',
       'АдресОрганизатора', 'Главный орган', 'IDOrganizator', 'Номер', 'Назва',
       'Тип закладу', 'Заклад-замовник', 'Correct'],
      dtype='object')


#### Step 11: RandomDataResuls

In [48]:
"""
Колонка Correct:
    1 - коректо визначено
    
    Некоректно:
        2 - номер визначений неправильно:
            
            21 - відсутній символ №
            22 - кілька символів №
            23 - без лапок
            24 - номер чи назва в колонці Описание Тендер
            25 - лапки не визначились
            26 - кілька лапок
        
        3 - неправильно тип закладу
"""

random_1000_results = pandas.read_csv('results/school-procurements-1000-random-marked.csv', index_col = 0, error_bad_lines=False, sep = ',', low_memory = False)
random_1000_results = random_1000_results.iloc[:, :-1]
print(random_1000_results.shape)

(1000, 8)


In [51]:
random_1000_results.Correct.value_counts()

1     830
3     103
25     23
24     13
22     11
!      10
21      5
23      4
26      1
Name: Correct, dtype: int64

In [50]:
random_1000_results.index.values.tolist()

[445840,
 166960,
 180001,
 61147,
 158417,
 275071,
 40284,
 176930,
 4337,
 202331,
 360939,
 455985,
 186623,
 408544,
 235070,
 44401,
 66817,
 24729,
 81540,
 21678,
 460842,
 1203,
 130137,
 75071,
 167465,
 420438,
 171551,
 5493,
 399095,
 511745,
 181156,
 432562,
 1684,
 91736,
 405957,
 185220,
 178921,
 435657,
 65459,
 173502,
 83176,
 209286,
 240218,
 50640,
 174505,
 433481,
 4134,
 451802,
 67390,
 137841,
 495154,
 117765,
 162138,
 458966,
 219312,
 193334,
 269199,
 110881,
 535210,
 485531,
 131310,
 522087,
 83159,
 487907,
 471247,
 459636,
 460843,
 78083,
 473744,
 173748,
 467993,
 424413,
 443017,
 459038,
 323163,
 268618,
 194432,
 196047,
 390996,
 32349,
 522481,
 507787,
 324797,
 46198,
 319528,
 193353,
 465318,
 283136,
 75516,
 171434,
 8411,
 51593,
 57931,
 39604,
 449685,
 202284,
 360867,
 175297,
 492817,
 51725,
 512593,
 466295,
 2206,
 421136,
 270427,
 77206,
 212462,
 531119,
 168624,
 238727,
 79710,
 303564,
 76857,
 397778,
 523500,
 205